In [326]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns ##ploting and visual
import scipy.stats as stat
import numpy as np
import nltk.corpus
#nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None 
import datetime

%matplotlib inline
from scipy import stats
from sklearn.metrics.pairwise import linear_kernel # for cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [327]:
df= pd.read_csv("games.csv")

In [328]:
df.shape

(18800, 6)

In [329]:
df.drop_duplicates('name',keep='first', inplace=True)

df.reset_index(drop=True, inplace=True)

In [330]:
df.shape

(12254, 6)

In [331]:
df = df[df["summary"].str.contains("spin-off|DLC|Expansion") == False]

In [332]:
df.shape

(12030, 6)

In [333]:
df.rename(columns={'summary': 's'}, inplace=True)
df['s']=df['s'].str.lower()
df['s'] = df['s'].str.replace(r'[^\w\s]+', '')
print(df['s'])


/var/folders/l8/z5jtbjgx7xg8pwl0g1mdd2fh0000gn/T/ipykernel_64054/1066207659.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['s'] = df['s'].str.replace(r'[^\w\s]+', '')


0        as a young boy link is tricked by ganondorf th...
1        as most major publishers development efforts s...
2        metacritics 2008 ps3 game of the year also kno...
3        this is a tale of souls and swords transcendin...
4        metacritics 2007 wii game of the year the ulti...
                               ...                        
12249    join natalie dylan and alex for an intense adv...
12250    fast  furious showdown takes some of the franc...
12251    drake is out for revenge in a supernatural hon...
12252    head out on a journey of redemption driven by ...
12253    formerly known as the war z it has been 5 year...
Name: s, Length: 12030, dtype: object


In [334]:
null_rows = df[df['s'].isnull()].index

df.drop(index=null_rows, inplace=True)

In [335]:
stop_words = stopwords.words('english')
df['s'] = df['s'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(df['s'])

0        young boy link tricked ganondorf king gerudo t...
1        major publishers development efforts shift num...
2        metacritics 2008 ps3 game year also known gta ...
3        tale souls swords transcending world history t...
4        metacritics 2007 wii game year ultimate ninten...
                               ...                        
12249    join natalie dylan alex intense adventure base...
12250    fast furious showdown takes franchises iconic ...
12251    drake revenge supernatural hong kong world you...
12252    head journey redemption driven bittersweet rev...
12253    formerly known war z 5 years since outbreak gi...
Name: s, Length: 12030, dtype: object


In [347]:
df2 = df.drop(df[df['user_review'] == 'tbd'].index)
df2["user_review"] = df["user_review"].astype(str).astype(float)
df2["meta_score"] = df["meta_score"].astype(str).astype(float)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7848 entries, 0 to 8693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          7848 non-null   object 
 1   platform      7848 non-null   object 
 2   release_date  7848 non-null   object 
 3   s             7848 non-null   object 
 4   meta_score    7848 non-null   float64
 5   user_review   7848 non-null   float64
dtypes: float64(2), object(4)
memory usage: 687.2+ KB


In [348]:
df3 = df2[~df.platform.str.contains("Dreamcast")]
df4 = df3[~df.platform.str.contains("PlayStation Vita")]

In [349]:
condition = df4['meta_score'] > 65
true_index = condition[condition == True].index
df5 =df4[condition]
df5

,name,platform,release_date,s,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",young boy link tricked ganondorf king gerudo t...,99.0,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",major publishers development efforts shift num...,98.0,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",metacritics 2008 ps3 game year also known gta ...,98.0,7.7
4,Super Mario Galaxy,Wii,"November 12, 2007",metacritics 2007 wii game year ultimate ninten...,97.0,9.1
5,Super Mario Galaxy 2,Wii,"May 23, 2010",super mario galaxy 2 sequel galaxyhopping orig...,97.0,9.1
...,...,...,...,...,...,...
8688,AirForce Delta Strike,PlayStation 2,"February 3, 2004",also known airforce delta blue wing knights st...,66.0,7.0
8689,American Conquest: Fight Back,PC,"September 29, 2003",three centuries war continue even frantic comb...,66.0,8.1
8690,Tomb Raider: The Prophecy,Game Boy Advance,"November 12, 2002",tomb raider prophecy delivers full tomb raider...,66.0,8.3
8691,Home Run King,GameCube,"March 18, 2002",first pitch opening day final world series hom...,66.0,7.5


In [351]:
df5.shape

(7848, 6)

In [361]:
df5.reset_index(drop=True, inplace=True)

df5.index

RangeIndex(start=0, stop=7848, step=1)

In [383]:
#First I did TF-IDF Vectorizer
tfidf = TfidfVectorizer()

#Created the matrix
tfidf_matrix = tfidf.fit_transform([str(i) for i in df['s']])

#Output the shape of matrix
tfidf_matrix.shape

(7848, 35011)

In [363]:
cosine = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine

array([[1.        , 0.        , 0.0227759 , ..., 0.        , 0.0274081 ,
        0.00709838],
       [0.        , 1.        , 0.00799464, ..., 0.        , 0.05309602,
        0.        ],
       [0.0227759 , 0.00799464, 1.        , ..., 0.        , 0.01036121,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.0274081 , 0.05309602, 0.01036121, ..., 0.        , 1.        ,
        0.        ],
       [0.00709838, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [364]:
# reverse the indices and video game titles

indices = pd.Series(df5.index, index=df5['name'])

indices

name
The Legend of Zelda: Ocarina of Time       0
Tony Hawk's Pro Skater 2                   1
Grand Theft Auto IV                        2
Super Mario Galaxy                         3
Super Mario Galaxy 2                       4
                                        ... 
AirForce Delta Strike                   7843
American Conquest: Fight Back           7844
Tomb Raider: The Prophecy               7845
Home Run King                           7846
Disney's The Emperor's New Groove       7847
Length: 7848, dtype: int64

In [365]:
def recommender_system(title, cosine=cosine):
    
    # get index of video game that matches title
    
    index = indices[title]
    
    # get pairwise similarity scores of all video games with the given title
    
    sim_scores = list(enumerate(cosine[index]))
    
    # sort games based on similarity scores
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # get scores of 10 most similar video games
    
    sim_scores = sim_scores[1:11]
    
    # get game indices
    
    game_indices = [i[0] for i in sim_scores]
    
    # return top 10 most similar video games
    
    recs = df5['name'].iloc[game_indices]
    
    return recs

In [366]:
recommender_system("Assassin's Creed II")

5836          Assassin's Creed: The Ezio Collection
6827                 Assassin's Creed II: Discovery
2633         Assassin's Creed: The Rebel Collection
5987    Assassin's Creed Syndicate: Jack the Ripper
6969             Assassin's Creed Chronicles: China
1123                       Assassin's Creed Origins
337                   Assassin's Creed: Brotherhood
2335                               Assassin's Creed
3091       Assassin's Creed: Director's Cut Edition
5608                           Creed: Rise to Glory
Name: name, dtype: object

In [367]:
recommender_system("Super Mario Galaxy")

2071                              Mario Kart Wii
727                    New Super Mario Bros. Wii
99                          Super Mario 3D World
6951                        Mario Super Sluggers
2003                    Super Mario 3D All-Stars
156     Super Mario World: Super Mario Advance 2
901                                  Punch-Out!!
2657                           Wii Sports Resort
3683                       Wario Land: Shake It!
4321                    Mario & Luigi: Paper Jam
Name: name, dtype: object

In [378]:
recommender_system("Far Cry 3")

6984    Friday The 13th: The Game - Ultimate Slasher S...
4278                       Friday the 13th: Killer Puzzle
1073                               Blaster Master Zero II
3601                           Far Cry Instincts Predator
2638                                          Dead Island
5400                          The Sims 3: Island Paradise
1375                                              Gorogoa
456                                               Far Cry
5358                           The Sims: Castaway Stories
1155                           Ys VIII: Lacrimosa of DANA
Name: name, dtype: object